In [3]:
import torch
import argparse
import os
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from transformers import AutoTokenizer, TrainingArguments
from mamba_trainer.data import DataModule
from mamba_trainer.trainer import MambaTrainer, GradientCallback

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,5,6'

model = MambaLMHeadModel.from_pretrained("state-spaces/mamba-1.4b", dtype=torch.bfloat16, device="cuda")
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer.eos_token = "<|endoftext|>"
tokenizer.pad_token = tokenizer.eos_token

optimizer = torch.optim.Adam(model.parameters(), amsgrad=True)

data_module = DataModule(
    tokenizer=tokenizer,
    data_path="./data/listops_500-1000/train.tsv",
    batch_size=64
)

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    gradient_accumulation_steps=1,
    output_dir="model",
    logging_dir="./logs", 
    logging_steps=1,
    save_steps=1,
    report_to="none", 
)

gradient_norms = []

for step in range(1000):
    grad_callback = GradientCallback()

    trainer = MambaTrainer(
        model=model,
        train_dataset=data_module.dataset,
        tokenizer=tokenizer,
        optimizers=(optimizer, None)
        args=training_args,
        data_collator=data_module.data_collator,
        callbacks=[grad_callback]
    )

    trainer.train()
    gradient_norms.append(grad_callback.gradient_norm)

DeferredCudaCallError: CUDA call failed lazily at initialization with error: device >= 0 && device < num_gpus INTERNAL ASSERT FAILED at "../aten/src/ATen/cuda/CUDAContext.cpp":50, please report a bug to PyTorch. device=4, num_gpus=

CUDA call was originally invoked at:

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/sapr7y/noisy_ssm/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/sapr7y/noisy_ssm/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/sapr7y/noisy_ssm/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/sapr7y/noisy_ssm/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py